# Fine tuning usando Fast AutoAugment - Implantação

- Esse componente utiliza modelos para classificação de imagens disponíveis na biblioteca [PyTorch](https://pytorch.org/) previamente treinados utilizando polices genéricas encontradas pelo artigo [Fast AutoAugment](https://github.com/kakaobrain/fast-autoaugment/tree/master/FastAutoAugment)

### **Em caso de dúvidas, consulte os [tutoriais da PlatIAgro](https://platiagro.github.io/tutorials/).**

## Declaração de Classe para Predições em Tempo Real

A tarefa de implantação cria um serviço REST para predições em tempo-real.<br>
Para isso você deve criar uma classe `Model` que implementa o método `predict`.

In [ ]:
!wget https://raw.githubusercontent.com/platiagro/tasks/feature/finetuning-autoaugment/tasks/cv-fast-autocv/checkpoint.py

In [ ]:
%%writefile Model.py
import joblib
import torch
import numpy as np
import cv2
import base64

from checkpoint import Checkpoint


class Model:
    def __init__(self):
        artifacts = joblib.load("/tmp/data/model.joblib")
        self.model_name = artifacts["model_name"]
        self.model_path = artifacts["model_path"]
        self.model_arch = artifacts["model_arch"]
        self.dataset = artifacts["dataset"]
        self.used_police = artifacts["model_police"]
    
    def process_image(self, image):
        """Prepare single image for inference"""
        
        img = cv2.resize(image, (224, 224))
        # Convert to numpy, transpose color dimension and normalize
        img = np.array(img).transpose((2, 0, 1)) / 255
        # Standardization
        means = np.array([0.485, 0.456, 0.406]).reshape((3, 1, 1))
        stds = np.array([0.229, 0.224, 0.225]).reshape((3, 1, 1))
        img = img - means
        img = img / stds
        img_tensor = torch.Tensor(img)

        return img_tensor
    
    def predict(self, X: np.ndarray, feature_names, meta=None):
        
        # Rodar inferencia em cpu
        device = torch.device("cpu")
        multi_gpu = False
        
        # Decode image
        im_bytes = base64.b64decode(X[0,0])
        im_arr = np.frombuffer(im_bytes, dtype=np.uint8)
        image = cv2.imdecode(im_arr, flags=cv2.IMREAD_COLOR)
        
        # Selecionar apenas a classe predita de maior probabilidade
        topk = 1
        checkpoint = Checkpoint(self.model_arch, multi_gpu)
        model = checkpoint.load_checkpoint(self.model_arch, self.model_path)

        # Converte imagem em tensor do pytorch
        img_tensor = self.process_image(image)
        img_tensor = img_tensor.view(1, 3, 224, 224).to(device)

        with torch.no_grad():
            model.eval()
            out = model(img_tensor)
            ps = torch.exp(out)

            topk, topclass = ps.topk(topk, dim=1)
            topk_index = topclass.cpu().numpy()[0]
            topk_prob = topk.cpu().numpy()[0]
            result = {"class_index": int(topk_index), "class_probability": float(topk_prob)}
        return result


## Teste do serviço REST

Crie um arquivo `contract.json` com os seguintes atributos:

- `features` : A lista de features em uma requisição.
- `targets` : A lista de valores retornados pelo método `predict`.

Cada `feature` pode conter as seguintes informações:

- `name` : nome da feature
- `ftype` : tipo da feature : **continuous** ou **categorical**
- `dtype` : tipo de dado : **FLOAT** ou **INT** : *obrigatório para ftype continuous*
- `range` : intervalo de valores numéricos : *obrigatório para ftype continuous*
- `values` : lista de valores categóricos : *obrigatório para ftype categorical*

Em seguida, utilize a função `test_deployment` do [SDK da PlatIAgro](https://platiagro.github.io/sdk/) para simular predição em tempo-real.<br>

In [ ]:
%%writefile contract.json
{
    "features": [
        {
            "name": "bee_image",
            "ftype": "categorical",
            "values": ["..."]
        }
    ],
    "targets": []
}

In [ ]:
from platiagro.deployment import test_deployment

test_deployment("contract.json")